# Importing the libraries

In [2]:
import numpy as np
import random
import tensorflow as tf
import os
import plotly.graph_objs as go

# Set experiment parameters and seeds for random operations.

In [3]:
# --- 1. Parametri di Simulazione di Base ---
BETA = 50  # Fattore di spreading (lunghezza della sequenza caotica per simbolo)
M = 4      # Numero di sottoportanti
X0_INITIAL = 0.9 # Stato iniziale per le mappe caotiche
SNR_TEST_RANGE_DB = np.arange(0, 21, 1)
SNR_TRAIN_RANGE_DB = [0, 20] # SNR range for training

# Dataset sizes (adjust as needed for computational resources)
NUM_SYMBOLS_TRAIN = 60000
NUM_SYMBOLS_VALIDATION = 20000
NUM_SYMBOLS_TEST_PER_SNR = 500000


CHANNEL_TYPE = 'awgn'
L_FADING = 1 # Parametro per Rayleigh fading

# --- 1. Impostazione del Seed Globale all'inizio dello script ---
# Questo è il punto chiave per la riproducibilità di TUTTO ciò che segue.
MASTER_RANDOM_SEED = 42
np.random.seed(MASTER_RANDOM_SEED)
random.seed(MASTER_RANDOM_SEED) # Imposta anche il seed per la libreria 'random' di Python se la usi
tf.random.set_seed(MASTER_RANDOM_SEED)
os.environ['PYTHONHASHSEED'] = str(MASTER_RANDOM_SEED) # Per operazioni basate su hash (es. ordine dei dizionari)
os.environ['TF_DETERMINISTIC_OPS'] = '1' # Forza operazioni deterministiche in TensorFlow 2.x

# Print the HW Specs.

In [4]:
print("--- Dettagli dell'Architettura Hardware della Sessione Colab ---\n")

# --- 1. Dettagli CPU ---
print("--- Dettagli CPU ---")
!lscpu
print("\n")

# --- 2. Dettagli RAM (Memoria) ---
print("--- Dettagli RAM (Memoria) ---")
!cat /proc/meminfo | grep MemTotal
print("\n")

# --- 3. Dettagli Spazio su Disco ---
print("--- Dettagli Spazio su Disco ---")
!df -h /
print("\n")

# --- 4. Dettagli Acceleratore Hardware (GPU/TPU) ---
print("--- Dettagli Acceleratore Hardware (GPU/TPU) ---")
try:
    tpu_address = os.environ.get('COLAB_TPU_ADDR')
    if tpu_address:
        resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu=tpu_address)
        tf.config.experimental_connect_to_cluster(resolver)
        tf.tpu.experimental.initialize_tpu_system(resolver)
        print(f"Tipo Acceleratore: TPU (indirizzo: {tpu_address})")
        print("Dispositivi TPU disponibili:")
        for device in tf.config.list_logical_devices('TPU'):
            print(f"  - {device.name}")
    else:
        gpus = tf.config.list_physical_devices('GPU')
        if gpus:
            print(f"Tipo Acceleratore: GPU")
            for gpu in gpus:
                print(f"  - Dispositivo GPU rilevato: {gpu.name}")
            print("\nDettagli GPU specifici (da `!nvidia-smi`):")
            !nvidia-smi
        else:
            print("Tipo Acceleratore: Nessuna GPU o TPU rilevata (in uso CPU)")

except Exception as e:
    print(f"Si è verificato un errore durante la rilevazione dell'acceleratore: {e}")
    print("Tentativo di rilevare i dispositivi TensorFlow standard:")
    devices = tf.config.list_logical_devices()
    if devices:
        for device in devices:
            print(f"  - Dispositivo rilevato: {device.name}, Tipo: {device.device_type}")
    else:
        print("Nessun dispositivo TensorFlow rilevato.")

print("\n--- Analisi Dettagli Hardware Completata ---")

--- Dettagli dell'Architettura Hardware della Sessione Colab ---

--- Dettagli CPU ---
Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          46 bits physical, 48 bits virtual
  Byte Order:             Little Endian
CPU(s):                   2
  On-line CPU(s) list:    0,1
Vendor ID:                GenuineIntel
  Model name:             Intel(R) Xeon(R) CPU @ 2.20GHz
    CPU family:           6
    Model:                79
    Thread(s) per core:   2
    Core(s) per socket:   1
    Socket(s):            1
    Stepping:             0
    BogoMIPS:             4399.99
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge m
                          ca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht sysc
                          all nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xt
                          opology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq
                           ssse3 fma cx16 pcid sse4_1 sse4

# Connect To Gdrive to store the datasets created.

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Plot the results.

In [6]:
def moving_average(x, w=8):
    """Calcola la media mobile semplice con finestra w"""
    return np.convolve(x, np.ones(w)/w, mode='valid')

In [7]:
results_dir = "/content/drive/MyDrive/GitHub/AWGN/results_otf_test"
ber_files = [f for f in os.listdir(results_dir) if f.endswith("_ber.npy")]

# Da qui in poi seleziona solo i migliori.
# Dunque la mia CNN_9 e 128
filtered_files = [
    f for f in ber_files
    if not f.startswith("CNN_") or "CNN_9-ksize_128-filters" in f
]

# Esempio: SNR reali usati (modifica se necessario)
snr_points = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20])

ber_data = {}
for f in ber_files:
    model_name = f.replace("_ber.npy", "")
    path = os.path.join(results_dir, f)
    ber = np.load(path)
    valid_ber = ber[~np.isnan(ber)]
    if len(valid_ber) > 0:
        ber_data[model_name] = {
            "ber": ber,
            "mean": np.mean(valid_ber)
        }

# Trova miglior modello globale
best_model = min(ber_data.items(), key=lambda x: x[1]["mean"])
best_model_name = best_model[0]

# Crea grafico interattivo
fig = go.Figure()

for model_name, info in ber_data.items():
    visibility = True if model_name == best_model_name else 'legendonly'
    x_vals = snr_points
    y_vals = moving_average(info['ber']) #smooth_with_spline(snr_points, info["ber"])
    fig.add_trace(go.Scatter(
        x=x_vals,
        y=y_vals,
        mode='lines+markers',
        name=model_name,
        visible=visibility,
        line=dict(width=3 if model_name == best_model_name else 1.5)
    ))

fig.update_layout(
    title="📉 Curve BER Interattive per Modelli CNN",
    xaxis_title="SNR (dB)",
    yaxis_title="Bit Error Rate (BER)",
    yaxis_type="log",
    yaxis=dict(
        range=[-5, 0],  # scala log da 1e-5 a 1e0
        tickvals=[1e0, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7, 1e-8],
        ticktext=["$10^0$", "$10^{-1}$", "$10^{-2}$", "$10^{-3}$", "$10^{-4}$", "$10^{-5}$","$10^{-6}$","$10^{-7}$","$10^{-8}$"],
        showgrid=True,
        gridcolor='lightgray',
        gridwidth=1,
        dtick="D1",                 # ticks a ogni potenza e anche intermedi (2,3…9)
        tickformat="power",         # visualizza come 10⁻¹, 10⁻², ecc.
        minor=dict(                 # attiva griglia per i minori
            showgrid=True,
            gridcolor='lightgray',
            gridwidth=1
        ),
    ),
    xaxis=dict(
        showgrid=True,
        gridcolor='lightgray',
        gridwidth=1,
        tickvals=snr_points
    ),
    hovermode="x unified",
    legend_title="Modelli",
    template="plotly_white"
)

